In [2]:
from collections import OrderedDict
from torch.optim import Adam
from torch import nn

from backbone import resnet18
from neck import FPN
from head import DBHead
from loss import DBLoss
from WarmupPolyLR import WarmupPolyLR

model = nn.Sequential(OrderedDict([
                                   ('backbone', resnet18(pretrained=True)),
                                   ('neck', FPN([64,128,256,512])),
                                   ('head', DBHead(256))
]))
criterion = DBLoss()
optimizer = Adam(model.parameters(), lr=0.001)
scheduler = WarmupPolyLR(optimizer=optimizer, warmup_epoch=3)

In [21]:
from datasets.ICDAR2015Dataset import ICDAR2015Dataset, ICDARCollectFN
from torchvision import transforms
import torch

def get_transforms(transforms_config):
    tr_list = []
    for item in transforms_config:
        if 'args' not in item:
            args = {}
        else:
            args = item['args']
        cls = getattr(transforms, item['type'])(**args)
        tr_list.append(cls)
    tr_list = transforms.Compose(tr_list)
    return tr_list
    
img_transfroms = get_transforms([{'args': {}, 'type': 'ToTensor'},
 {'args': {'mean': [0.485, 0.456, 0.406], 'std': [0.229, 0.224, 0.225]},
  'type': 'Normalize'}])

data = ICDAR2015Dataset(['data/streets/train.txt'], 
                        img_mode= 'RGB',
                        filter_keys=['img_path',
                                      'img_name',
                                      'text_polys',
                                      'texts',
                                      'ignore_tags',
                                      'shape'],
                        transform=img_transfroms, 
                        ignore_tags= ['*', '###'], 
                        pre_processes=[{'args': [{'args': {'p': 0.5}, 'type': 'Fliplr'},
                                                 {'args': {'rotate': [-10, 10]}, 'type': 'Affine'},
                                                 {'args': {'size': [0.5, 3]}, 'type': 'Resize'}], 'type': 'IaaAugment'},
                                       {'args': {'keep_ratio': True, 'max_tries': 50, 'size': [640, 640]},
                                        'type': 'EastRandomCropData'},
                                       {'args': {'shrink_ratio': 0.4, 'thresh_max': 0.7, 'thresh_min': 0.3},
                                        'type': 'MakeBorderMap'},
                                       {'args': {'min_text_size': 8, 'shrink_ratio': 0.4}, 'type': 'MakeShrinkMap'}])


data_validation = ICDAR2015Dataset(['data/streets/test.txt'],img_mode= 'RGB',filter_keys=[],
                                   ignore_tags= ['*', '###'], 
                                   transform=img_transfroms,
                                   pre_processes=[
                                       {'args': {'shrink_ratio': 0.4, 'thresh_max': 0.7, 'thresh_min': 0.3},
                                        'type': 'MakeBorderMap'},
                                       {'args': {'min_text_size': 8, 'shrink_ratio': 0.4}, 'type': 'MakeShrinkMap'}],)

In [30]:
from torch.utils.data import DataLoader
loader = DataLoader(dataset=data, batch_size=1, shuffle=True, num_workers=0, pin_memory=True)
loader_validation = DataLoader(dataset=data_validation, batch_size=1, shuffle=True, num_workers=6, pin_memory=False)

In [40]:
from time import time
import numpy as np
from tqdm import tqdm


device = 'cuda'
BATCH_SIZE=1
def train_epoch(epoch):
    model.to(device)
    model.train()
    epoch_start = time()
    

    for i, batch in tqdm(enumerate(loader), total=len(loader), desc='train epoch {}'.format(epoch)):
        if i >= len(data)/BATCH_SIZE:
            break
        batch_start = time()

        for key, value in batch.items():
            if value is not None:
                if isinstance(value, torch.Tensor):
                    batch[key] = value.to(device)

        preds = model(batch['img'])
        loss_dict = criterion(preds, batch)
        # backward
        optimizer.zero_grad()
        loss_dict['loss'].backward()
        optimizer.step()
        scheduler.step()
        #if (i+1)%100==0:
            #print('{loss:5}---{loss_binary_maps:5}---{loss_shrink_maps:5}---{loss_threshold_maps:5}'.format(**loss_dict))
            #print('EPOCH--{}, --{}/{}batch_time:'.format(epoch, i,len(data)/BATCH_SIZE), time() - batch_start)

    print('time', time() - epoch_start)
    
    
def validate(epoch):
    model.to(device)
    model.train()
    losses = []
    with torch.no_grad():
        for i, batch in tqdm(enumerate(loader_validation), total=len(loader_validation), desc='test model'):
            if i >= len(data_validation)/BATCH_SIZE:
                break
            for key, value in batch.items():
                if value is not None:
                    if isinstance(value, torch.Tensor):
                        batch[key] = value.to(device)
            preds = model(batch['img'])
            loss_dict = criterion(preds, batch)
            losses.append(loss_dict['loss'])
    print('VALIDATION LOSSES ON EPOCH {}:'.format(epoch), torch.tensor(losses).mean())
    return torch.tensor(losses).mean()

In [ ]:
from time import time
epochs = 1000

min_val_loss = 1e6

for epoch in range(epochs):
    eptimest=time()
    train_epoch(epoch)
    eptime = time()-eptimest
    epmin = eptime//60
    epsec = round(eptime%60)
    print(epoch, "EPOCH TIME: {epmin:3} mins {epsec:2} sec".format(epmin=epmin, epsec=epsec))
    val_loss = validate(epoch)
    if val_loss < min_val_loss:
        print('NEW BEST!')
        torch.save(model.state_dict(), 'best_model.pth')
        min_val_loss = val_loss        
    torch.save(model.state_dict(), 'latest_model.pth')

train epoch 0: 100%|██████████| 1000/1000 [01:29<00:00, 11.16it/s]

time 89.604421377182
0 EPOCH TIME: 1.0 mins 30 sec



test model: 100%|██████████| 500/500 [00:37<00:00, 13.20it/s]


VALIDATION LOSSES ON EPOCH 0: tensor(1.7949)
NEW BEST!


train epoch 1: 100%|██████████| 1000/1000 [01:27<00:00, 11.37it/s]

...

train epoch 220: 100%|██████████| 1000/1000 [01:28<00:00, 11.32it/s]

time 88.36470413208008
220 EPOCH TIME: 1.0 mins 28 sec



test model: 100%|██████████| 500/500 [00:38<00:00, 13.04it/s]


VALIDATION LOSSES ON EPOCH 220: tensor(1.5652)


train epoch 221:  96%|█████████▌| 960/1000 [01:24<00:03, 10.48it/s]IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

train epoch 224:  66%|██████▌   | 656/1000 [00:58<00:29, 11.56it/s]

In [1]:
import torch
torch.cuda.is_available()

True

In [27]:
torch.tensor(losses).mean()

tensor(16.0764)

In [ ]:
'dsadsads'.в